In [2]:
import os
from anthropic import AsyncAnthropic
from tqdm import tqdm

In [3]:
def link_to_file_name(link):
    return link[39:].replace('/', '.')

In [ ]:
INPUT_DIR = "page_content"

SYSTEM = """
You are an agent that is an expert at Manim, a Python library that can be compiled to create video tutorials for educational materials. Your task is to take in a textbook chapter covering some material and create an educational tutorial using Manim, detailing technical parts of the textbook to make it intuitive. The idea is to have different sections covering the topic, with visualizations of mathematical concepts and explanatory text as needed. 

First, understand the topic at hand. If there are multiple, focus on the first one and then iterate through the rest. Then, understand how the textbook conveys material and equations. If there is a lot of material to cover, break it up into sections, processing each section at a time. Make it such that a beginner new to this field understands it.

Ensure that your Manim tutorial goes in depth to each of these topics, creating a detailed video of at least a minute explaining the topic, having an example, and ending in a summary. Compile all the events in a term called FullTutorial. Additionally, if you need to include equations, ensure that you write it in LaTeX that can be compiled accurately. It is essential that you write safe LaTeX that only uses valid characters and formatting such that there are no issues with it. Use other materials like graphs or plots as well. When writing the steps of each topic using the Manim Community library in Python, ensuring that your code works correctly. Output a valid solution that can be run, producing a correct video without any errors whatsoever. Ensure that the text all fits in the screen and does not overlap with one another. I will provide several example input outputs to you as well initially.

===

You will be provided the full documentation of the Manim. You can find the full set of page links and associated summary of the given page of documentation below. Then, to view the actual details of some given documentation you can call the given function 
"""

In [ ]:
for file_name in tqdm(os.listdir(INPUT_DIR)):
    path = os.path.join(INPUT_DIR, file_name)
    
    with open(path, "rb") as f:
        url = f.readline().decode('utf-8').strip()
    
    content = extract_text_and_media(path)
    user_message = [construct_message(url, content)]
    
    query = client.messages.create(
        model="claude-3-5-haiku-20241022",
        temperature=1,
        max_tokens=8192,
        system="You are an expert technical writer. You will be given the content of some documentation for the Manim library. Please summarize the information you are given but do so such that someone with no experience can still get the same amount of information from the summary (feel free to include the examples) but also try to keep it as short as possible to minimize token use (pretty important). Also, if you believe the information is not relevant or important for an average user to code in Manim, you can simply say 'Not relevant' -- keep your standards high, for example something like the changelog or how to contribute isn't important. Just to reiterate, unimportant things like the changelog should not be summarized and you should just say 'Not relevant' -- really make sure that only the actually absolute necessary things in the documentation are summarized. Only include your reponse/summary and absolutely nothing more, don't have any unnecessary comments (since minimizing response length is really important). To reiterate again, make the response as short as possible!",
        messages=user_message
    )
    results.append((file_name, query))